In [ ]:
!pip install datasets transformers nltk evaluate rouge_score transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 12.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-1

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

# First try - t5-small

Model: t5-small
Hyperparameters:
* Batch size: 8
* Learning rate: 2e-5

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from datasets import load_dataset
dataset = load_dataset('AlexanderBenady/generated_lectures')
metric = evaluate.load("rouge")
model_checkpoint = "t5-small" # "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1100
max_target_length = 250

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["lectures"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
data = dataset["train"]
datax = data["Lecture"]
datay = data["Summary"]

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the data into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(datax, datay, test_size=0.2, random_state=42)

# Further split the test set into test and validation sets
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=42)

# Convert train_features and train_labels into a dictionary
train_data_dict = {'lectures': train_features, 'summary': train_labels}
# Convert test_features and test_labels into a dictionary
test_data_dict = {'lectures': test_features, 'summary': test_labels}
# Convert val_features and val_labels into a dictionary
val_data_dict = {'lectures': val_features, 'summary': val_labels}

# Create datasets from the dictionaries
trainset = Dataset.from_dict(train_data_dict)
testset = Dataset.from_dict(test_data_dict)
valset = Dataset.from_dict(val_data_dict)


In [ ]:
tokenized_dataset_train = trainset.map(preprocess_function, batched=True)
tokenized_dataset_test = testset.map(preprocess_function, batched=True)
tokenized_dataset_val = valset.map(preprocess_function, batched=True)

Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Fine tuning the Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=2
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.624091,15.575000,6.655200,12.895500,14.370500,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=50, training_loss=3.489609375, metrics={'train_runtime': 36.5651, 'train_samples_per_second': 21.851, 'train_steps_per_second': 1.367, 'total_flos': 230122035118080.0, 'train_loss': 3.489609375, 'epoch': 1.0})

# Evaluate with BART Score

In [ ]:
import torch
import torch.nn as nn
import traceback
from transformers import BartTokenizer, BartForConditionalGeneration
from typing import List

In [ ]:
class BARTScorer:
    def __init__(self, device='cuda:0', max_length=1024, checkpoint='facebook/bart-large-cnn'):
        # Set up model
        self.device = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

    def load(self, path=None):
        """ Load model from paraphrase finetuning """
        if path is None:
            path = 'models/bart.pth'
        self.model.load_state_dict(torch.load(path, map_location=self.device))

    def score(self, srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        for i in range(0, len(srcs), batch_size):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            try:
                with torch.no_grad():
                    encoded_src = self.tokenizer(
                        src_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = self.tokenizer(
                        tgt_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(self.device)
                    src_mask = encoded_src['attention_mask'].to(self.device)

                    tgt_tokens = encoded_tgt['input_ids'].to(self.device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(self.device)

                    output = self.model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, self.model.config.vocab_size)
                    loss = self.loss_fct(self.lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    loss = loss.sum(dim=1) / tgt_len
                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list

    def multi_ref_score(self, srcs, tgts: List[List[str]], agg="mean", batch_size=4):
        # Assert we have the same number of references
        ref_nums = [len(x) for x in tgts]
        if len(set(ref_nums)) > 1:
            raise Exception("You have different number of references per test sample.")

        ref_num = len(tgts[0])
        score_matrix = []
        for i in range(ref_num):
            curr_tgts = [x[i] for x in tgts]
            scores = self.score(srcs, curr_tgts, batch_size)
            score_matrix.append(scores)
        if agg == "mean":
            score_list = np.mean(score_matrix, axis=0)
        elif agg == "max":
            score_list = np.max(score_matrix, axis=0)
        else:
            raise NotImplementedError
        return list(score_list)

    def test(self, batch_size=3):
        """ Test """
        src_list = [
            'This is a very good idea. Although simple, but very insightful.',
            'Can I take a look?',
            'Do not trust him, he is a liar.'
        ]

        tgt_list = [
            "That's stupid.",
            "What's the problem?",
            'He is trustworthy.'
        ]

        print(self.score(src_list, tgt_list, batch_size))

In [ ]:
bart_scorer = BARTScorer(device="cuda:0", checkpoint='facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def evaluate_func(original_outputs, predicted_outputs):
        precision_scores = bart_scorer.score(original_outputs, predicted_outputs, batch_size=16) # r -> h
        recall_scores = bart_scorer.score(predicted_outputs, original_outputs, batch_size=16) # h -> r
        fscores = [(p + r) / 2 for p, r in zip(precision_scores, recall_scores)]
        fscore = sum(fscores) / len(fscores)
        return {"bartscore": fscore}

In [ ]:
preds = trainer.predict(tokenized_dataset_test, num_beams=4)

In [ ]:
predictions = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

In [ ]:
predictions

['thermometers are devices used to measure temperature, a key parameter in weather analysis.',
 'neurons are the building blocks of the nervous system, responsible for transmitting electrical and chemical',
 'entomologists are like detectives, unraveling the mysteries of insect behavior, anatomy',
 'game development is a multifaceted process that involves creativity, technical skills, and effective project management',
 'drug design is the process of creating new medications based on a thorough understanding of the biological',
 'the politics of development in the Global South are complex and multifaceted, shaped by a',
 'logistics is the process of planning, implementing, and controlling the movement and storage of goods,',
 'parametric and computational design in architecture is revolutionizing the way architects approach design, enabling the',
 'Expressionism was a movement that emerged in the early 20th century, particularly in Germany ',
 'strategic HRM is all about integrating HR

In [ ]:
testset

Dataset({
    features: ['lectures', 'summary'],
    num_rows: 100
})

In [ ]:
len(testset["summary"])

100

In [ ]:
metrics = evaluate_func(testset["summary"], predictions)

In [ ]:
metrics

{'bartscore': -2.9923841941356657}

In [ ]:
from datasets import load_metric

# Load the ROUGE metric
rouge_metric = load_metric("rouge")
references = testset["summary"]
# Assuming your predictions and references are stored in lists
# Calculate ROUGE scores
rouge_scores = rouge_metric.compute(predictions=predictions, references=references)

# Print ROUGE scores
print(rouge_scores)

<ipython-input-27-93645aad0d2b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.6081101470629042, recall=0.0814172943054923, fmeasure=0.14345710814522125), mid=Score(precision=0.6409181446563414, recall=0.0863870693784309, fmeasure=0.1518776494803172), high=Score(precision=0.6750781566472743, recall=0.09152312006685441, fmeasure=0.16067941036152616)), 'rouge2': AggregateScore(low=Score(precision=0.24885859650969938, recall=0.03136407635045181, fmeasure=0.05563225605164842), mid=Score(precision=0.2892832041324688, recall=0.03679911772151819, fmeasure=0.06520519700767821), high=Score(precision=0.3320543481028775, recall=0.04242221164600444, fmeasure=0.07494113451766907)), 'rougeL': AggregateScore(low=Score(precision=0.4811787734077497, recall=0.06443127198748251, fmeasure=0.11345286546639768), mid=Score(precision=0.5172449143134126, recall=0.06959320017555151, fmeasure=0.12231144196496896), high=Score(precision=0.5521072371009192, recall=0.07469651235676085, fmeasure=0.13113183700919862)), 'rougeLsum': AggregateScore(l

As seen in class, BARTScore is not constrained between 0 and 1, but can give any value. The metric is useful for comparing different summarization models for the given dataset, which is the actual objective of a metric. But we cannot get a general idea of how this model works without comparing the metric against another model.

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 2.6240906715393066,
 'eval_rouge1': 15.575,
 'eval_rouge2': 6.6552,
 'eval_rougeL': 12.8955,
 'eval_rougeLsum': 14.3705,
 'eval_gen_len': 19.0,
 'eval_runtime': 7.5302,
 'eval_samples_per_second': 13.28,
 'eval_steps_per_second': 1.726,
 'epoch': 1.0}

In [ ]:
# trainer.push_to_hub(commit_message="Training complete", tags="summarization")

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

# Train with OPTUNA HYPERPARAMETER TUNING

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

nltk.download("punkt")

from datasets import load_dataset
dataset = load_dataset('AlexanderBenady/generated_lectures')
metric = evaluate.load("rouge")
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1100
max_target_length = 250

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["lectures"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data = dataset["train"]
datax = data["Lecture"]
datay = data["Summary"]

from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the data into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(datax, datay, test_size=0.2, random_state=42)

# Further split the test set into test and validation sets
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=42)

# Convert train_features and train_labels into a dictionary
train_data_dict = {'lectures': train_features, 'summary': train_labels}
# Convert test_features and test_labels into a dictionary
test_data_dict = {'lectures': test_features, 'summary': test_labels}
# Convert val_features and val_labels into a dictionary
val_data_dict = {'lectures': val_features, 'summary': val_labels}

# Create datasets from the dictionaries
trainset = Dataset.from_dict(train_data_dict)
testset = Dataset.from_dict(test_data_dict)
valset = Dataset.from_dict(val_data_dict)

tokenized_dataset_train = trainset.map(preprocess_function, batched=True)
tokenized_dataset_test = testset.map(preprocess_function, batched=True)
tokenized_dataset_val = valset.map(preprocess_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=2
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

import torch
import torch.nn as nn
import traceback
from transformers import BartTokenizer, BartForConditionalGeneration
from typing import List

class BARTScorer:
    def __init__(self, device='cuda:0', max_length=1024, checkpoint='facebook/bart-large-cnn'):
        # Set up model
        self.device = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

    def load(self, path=None):
        """ Load model from paraphrase finetuning """
        if path is None:
            path = 'models/bart.pth'
        self.model.load_state_dict(torch.load(path, map_location=self.device))

    def score(self, srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        for i in range(0, len(srcs), batch_size):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            try:
                with torch.no_grad():
                    encoded_src = self.tokenizer(
                        src_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = self.tokenizer(
                        tgt_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(self.device)
                    src_mask = encoded_src['attention_mask'].to(self.device)

                    tgt_tokens = encoded_tgt['input_ids'].to(self.device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(self.device)

                    output = self.model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, self.model.config.vocab_size)
                    loss = self.loss_fct(self.lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    loss = loss.sum(dim=1) / tgt_len
                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list

    def multi_ref_score(self, srcs, tgts: List[List[str]], agg="mean", batch_size=4):
        # Assert we have the same number of references
        ref_nums = [len(x) for x in tgts]
        if len(set(ref_nums)) > 1:
            raise Exception("You have different number of references per test sample.")

        ref_num = len(tgts[0])
        score_matrix = []
        for i in range(ref_num):
            curr_tgts = [x[i] for x in tgts]
            scores = self.score(srcs, curr_tgts, batch_size)
            score_matrix.append(scores)
        if agg == "mean":
            score_list = np.mean(score_matrix, axis=0)
        elif agg == "max":
            score_list = np.max(score_matrix, axis=0)
        else:
            raise NotImplementedError
        return list(score_list)

    def test(self, batch_size=3):
        """ Test """
        src_list = [
            'This is a very good idea. Although simple, but very insightful.',
            'Can I take a look?',
            'Do not trust him, he is a liar.'
        ]

        tgt_list = [
            "That's stupid.",
            "What's the problem?",
            'He is trustworthy.'
        ]

        print(self.score(src_list, tgt_list, batch_size))



bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')

def evaluate_func(original_outputs, predicted_outputs):
        precision_scores = bart_scorer.score(original_outputs, predicted_outputs, batch_size=16) # r -> h
        recall_scores = bart_scorer.score(predicted_outputs, original_outputs, batch_size=16) # h -> r
        fscores = [(p + r) / 2 for p, r in zip(precision_scores, recall_scores)]
        fscore = sum(fscores) / len(fscores)
        return {"bartscore": fscore}

preds = trainer.predict(tokenized_dataset_test, num_beams=4)

predictions = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

print(predictions)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.624091,15.612400,6.642000,12.854800,14.381900,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['thermometers are devices used to measure temperature, a key parameter in weather analysis.', 'neurons are the building blocks of the nervous system, responsible for transmitting electrical and chemical', 'entomologists are like detectives, unraveling the mysteries of insect behavior, anatomy', 'game development is a multifaceted process that involves creativity, technical skills, and effective project management', 'drug design is the process of creating new medications based on a thorough understanding of the biological', 'the politics of development in the Global South are complex and multifaceted, shaped by a', 'logistics is the process of planning, implementing, and controlling the movement and storage of goods,', 'parametric and computational design in architecture is revolutionizing the way architects approach design, enabling the', 'Expressionism was a movement that emerged in the early 20th century, particularly in Germany ', 'strategic HRM is all about integrating HR practice

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk
import optuna

nltk.download("punkt")

def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-7, 1e-4)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-7, 1e-4)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1,  5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])

    args = Seq2SeqTrainingArguments(
        f"{model_name}-finetuned-xsum",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        save_total_limit=3,
        num_train_epochs=num_train_epochs,
        predict_with_generate=True,
        push_to_hub=False,
        gradient_accumulation_steps=2
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    preds = trainer.predict(tokenized_dataset_test, num_beams=4)
    predictions = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

    # Evaluation
    fscore = evaluate_func(test_labels, predictions)['bartscore']

    return fscore

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # You can adjust n_trials as needed
best_params = study.best_params
print("Best hyperparameters:", best_params)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[I 2024-04-10 10:27:43,924] A new study created in memory with name: no-name-30af1fe0-ff90-4922-b9d8-b5275f70ad2d
<ipython-input-36-1a035ab66074>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-7, 1e-4)
<ipython-input-36-1a035ab66074>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-7, 1e-4)
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated an

OutOfMemoryError: CUDA out of memory. Tried to allocate 592.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 386.38 MiB is free. Process 16643 has 15.39 GiB memory in use. Of the allocated memory 14.79 GiB is allocated by PyTorch, and 237.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.evaluate()

# PEGASUS DEBUG

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

import torch
torch.autograd.set_detect_anomaly(True)

nltk.download("punkt")

from datasets import load_dataset
dataset = load_dataset('AlexanderBenady/generated_lectures')
data = dataset["train"]
datax = data["Lecture"]
datay = data["Summary"]

metric = evaluate.load("rouge")
model_checkpoint = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if model_checkpoint in ["t5-small"]: # "t5-base","t5-large", "t5-3b", "t5-11b"
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["lectures"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

from sklearn.model_selection import train_test_split
from datasets import Dataset
from torch.utils.data import DataLoader

# Split the data into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(datax, datay, test_size=0.2, random_state=42)

# Further split the test set into test and validation sets
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=42)

# Convert train_features and train_labels into a dictionary
train_data_dict = {'lectures': train_features, 'summary': train_labels}
# Convert test_features and test_labels into a dictionary
test_data_dict = {'lectures': test_features, 'summary': test_labels}
# Convert val_features and val_labels into a dictionary
val_data_dict = {'lectures': val_features, 'summary': val_labels}

# Create datasets from the dictionaries
trainset = Dataset.from_dict(train_data_dict)
testset = Dataset.from_dict(test_data_dict)
valset = Dataset.from_dict(val_data_dict)

tokenized_dataset_train = trainset.map(preprocess_function, batched=True)
tokenized_dataset_val = valset.map(preprocess_function, batched=True)
tokenized_dataset_test = testset.map(preprocess_function, batched=True)

batch_size = 2 # CHANGE TO 8

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Recreate DataLoaders with the updated datasets
train_loader = DataLoader(tokenized_dataset_train, batch_size=batch_size, collate_fn=data_collator, shuffle=True)
val_loader = DataLoader(tokenized_dataset_val, batch_size=batch_size, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset_test, batch_size=batch_size, collate_fn=data_collator)

train_loader = DataLoader(tokenized_dataset_train, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset_val, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset_test, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

batch_size = 2 # CHANGE TO 8
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Recreate DataLoaders with the updated datasets
train_loader = DataLoader(tokenized_dataset_train, batch_size=batch_size, collate_fn=data_collator, shuffle=True)
val_loader = DataLoader(tokenized_dataset_val, batch_size=batch_size, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset_test, batch_size=batch_size, collate_fn=data_collator)

train_loader = DataLoader(tokenized_dataset_train, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset_val, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset_test, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=2
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in NativeDropoutBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp

RuntimeError: Function 'NativeDropoutBackward0' returned nan values in its 0th output.

# Train with Hyper parameter tuning for Pegasus

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

nltk.download("punkt")

from datasets import load_dataset
dataset = load_dataset('AlexanderBenady/generated_lectures')
metric = evaluate.load("rouge")
model_checkpoint = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["lectures"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data = dataset["train"]
datax = data["Lecture"]
datay = data["Summary"]

from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the data into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(datax, datay, test_size=0.2, random_state=42)

# Further split the test set into test and validation sets
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=42)

# Convert train_features and train_labels into a dictionary
train_data_dict = {'lectures': train_features, 'summary': train_labels}
# Convert test_features and test_labels into a dictionary
test_data_dict = {'lectures': test_features, 'summary': test_labels}
# Convert val_features and val_labels into a dictionary
val_data_dict = {'lectures': val_features, 'summary': val_labels}

# Create datasets from the dictionaries
trainset = Dataset.from_dict(train_data_dict)
testset = Dataset.from_dict(test_data_dict)
valset = Dataset.from_dict(val_data_dict)

tokenized_dataset_train = trainset.map(preprocess_function, batched=True)
tokenized_dataset_test = testset.map(preprocess_function, batched=True)
tokenized_dataset_val = valset.map(preprocess_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=2
)

device = torch.device("cpu")
model.to(device)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 376.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 10.38 MiB is free. Process 9735 has 15.76 GiB memory in use. Of the allocated memory 15.37 GiB is allocated by PyTorch, and 24.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
import torch.nn as nn
import traceback
from transformers import BartTokenizer, BartForConditionalGeneration
from typing import List

class BARTScorer:
    def __init__(self, device='cuda:0', max_length=1024, checkpoint='facebook/bart-large-cnn'):
        # Set up model
        self.device = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

    def load(self, path=None):
        """ Load model from paraphrase finetuning """
        if path is None:
            path = 'models/bart.pth'
        self.model.load_state_dict(torch.load(path, map_location=self.device))

    def score(self, srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        for i in range(0, len(srcs), batch_size):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            try:
                with torch.no_grad():
                    encoded_src = self.tokenizer(
                        src_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = self.tokenizer(
                        tgt_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(self.device)
                    src_mask = encoded_src['attention_mask'].to(self.device)

                    tgt_tokens = encoded_tgt['input_ids'].to(self.device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(self.device)

                    output = self.model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, self.model.config.vocab_size)
                    loss = self.loss_fct(self.lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    loss = loss.sum(dim=1) / tgt_len
                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list

    def multi_ref_score(self, srcs, tgts: List[List[str]], agg="mean", batch_size=4):
        # Assert we have the same number of references
        ref_nums = [len(x) for x in tgts]
        if len(set(ref_nums)) > 1:
            raise Exception("You have different number of references per test sample.")

        ref_num = len(tgts[0])
        score_matrix = []
        for i in range(ref_num):
            curr_tgts = [x[i] for x in tgts]
            scores = self.score(srcs, curr_tgts, batch_size)
            score_matrix.append(scores)
        if agg == "mean":
            score_list = np.mean(score_matrix, axis=0)
        elif agg == "max":
            score_list = np.max(score_matrix, axis=0)
        else:
            raise NotImplementedError
        return list(score_list)

    def test(self, batch_size=3):
        """ Test """
        src_list = [
            'This is a very good idea. Although simple, but very insightful.',
            'Can I take a look?',
            'Do not trust him, he is a liar.'
        ]

        tgt_list = [
            "That's stupid.",
            "What's the problem?",
            'He is trustworthy.'
        ]

        print(self.score(src_list, tgt_list, batch_size))



bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')

def evaluate_func(original_outputs, predicted_outputs):
        precision_scores = bart_scorer.score(original_outputs, predicted_outputs, batch_size=16) # r -> h
        recall_scores = bart_scorer.score(predicted_outputs, original_outputs, batch_size=16) # h -> r
        fscores = [(p + r) / 2 for p, r in zip(precision_scores, recall_scores)]
        fscore = sum(fscores) / len(fscores)
        return {"bartscore": fscore}

preds = trainer.predict(tokenized_dataset_test, num_beams=4)

predictions = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

print(predictions)

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk
import optuna

nltk.download("punkt")

def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-7, 1e-4)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-7, 1e-4)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1,  5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])

    args = Seq2SeqTrainingArguments(
        f"{model_name}-finetuned-xsum",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        save_total_limit=3,
        num_train_epochs=num_train_epochs,
        predict_with_generate=True,
        push_to_hub=False,
        gradient_accumulation_steps=2
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    preds = trainer.predict(tokenized_dataset_test, num_beams=4)
    predictions = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

    # Evaluation
    fscore = evaluate_func(test_labels, predictions)['bartscore']

    return fscore

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # You can adjust n_trials as needed
best_params = study.best_params
print("Best hyperparameters:", best_params)
